## MISO Linear Regression

**In order to see the results you should start the MLflow ui separately**:
1. Open a prompt/terminal and navigate to the path of this project
2. Activate the virtual environment:  
    (Windows: ```.venv\eis_data_analytics\Scripts\activate```,  
    Linux/Mac: ```.venv/eis_data_analytics/bin/activate```)
3. Now start MLflow with ```mlflow server --port 1234``` consider to add e.g.: ```--workers=16 --gunicorn-opts='--timeout 600'```
4. Open [http://127.0.0.1:1234](http://127.0.0.1:1234) in your browser

In [ ]:
%matplotlib widget 

from modules import dataset_manipulation as dm
from modules import eisplot as eisplot
from modules.eisplot import plt
from modules.eisplot import mpl

import numpy as np
import pandas as pd

import mlflow
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx.common.shape_calculator import calculate_linear_regressor_output_shapes
import onnxruntime as rt

## if you have installed latex and want to use it for plots, uncomment the following 3 lines
# mpl.rcParams.update({"text.usetex": True,'savefig.format':'pdf'})
# mpl.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
# mpl.rc('text.latex', preamble=r'\usepackage{underscore}')

## save figures e.g. with:
# plot_name = "custom_3D_plot"
# plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".pdf")
# plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".png", dpi=600)

### Load Data

In [ ]:
name_of_this_run = "example_data"

destination_filepath = r"./data/eis_datasets/" + name_of_this_run + ".parquet"
df = pd.read_parquet(destination_filepath)
destination_filepath = r"./data/key_lookup/key_lookup_" + name_of_this_run + ".parquet"
key_lookup_df = pd.read_parquet(destination_filepath)

### Define Input Parameters

In [ ]:
# Can be any of the following:
pd.set_option("display.max_columns", None)
df.head(0)

In [ ]:
# # Example of one frequency with polynomial features
# input_parameters = []
# frequency = 100
# input_parameters += [key_lookup_df["EIS_Z_abs"].loc[np.argmin(
#     np.abs(key_lookup_df["frequency"].values - frequency))]]
# # Give it a Name (filename friendly)
# input_parameters_name = "Z_abs_100hzPoly5"
# polynomial_degree = 5

In [ ]:
input_parameters = []
input_parameters += key_lookup_df["EIS_Z_abs"].to_list()
# input_parameters += key_lookup_df["EIS_Z_phase"].to_list()
# input_parameters.append("Voltage")
# Give it a Name (filename friendly)
input_parameters_name = "Z_abs"

# add polynomial features if desired, otherwise set to 1
polynomial_degree = 1

### Define Output Parameters

In [ ]:
# output_parameters = []
# output_parameters += ["SOC"]
# # Give it a Name (filename friendly)
# output_parameters_name = "SOC"

In [ ]:
# output_parameters = []
# output_parameters += ["SOH"]
# # Give it a Name (filename friendly)
# output_parameters_name = "SOH"

In [ ]:
output_parameters = []
output_parameters += ["Temperature"]
# Give it a Name (filename friendly)
output_parameters_name = "Temperature"

### Split up the data for training

In [ ]:
test_labels = ["Temperature"]
output_intervals_for_test = {"Temperature": [[4, 6], [34, 36]]}

# convert to training arrays
data_set = dm.get_set(
    df,
    output_parameters,
    feature_keys=input_parameters,
    validation_split=0.2,
    output_intervals_for_test=output_intervals_for_test,
    label_for_test_intervals=test_labels,
    label_name=output_parameters_name,
)
x_train, y_train = data_set["train"]
x_validation, y_validation = data_set["validation"]
x_test, y_test = data_set["test"]

# convert everything to float32
x_train = np.float32(x_train)
y_train = np.float32(y_train)
x_validation = np.float32(x_validation)
y_validation = np.float32(y_validation)
x_test = np.float32(x_test)
y_test = np.float32(y_test)

### Scale the data

In [ ]:
dm.use_arrhenius_correction = True
dm.use_arrhenius_correction_with_factor = False
dm.arrhenius_b = -15.47
dm.arrhenius_c = 1.30
# [dm.arrhenius_correction_inverse(dm.arrhenius_correction(i))
#  for i in [0.01, 0.1, 1, 10]]

In [ ]:
if dm.use_arrhenius_correction:
    x_train = dm.arrhenius_correction(x_train)
    x_validation = dm.arrhenius_correction(x_validation)
    x_test = dm.arrhenius_correction(x_test)

#### Add Polynomial Features

In [ ]:
input_parameters_backup = input_parameters.copy()
if polynomial_degree > 1:
    poly = PolynomialFeatures(
        degree=polynomial_degree, include_bias=False, interaction_only=False
    )
    x_train = poly.fit_transform(x_train)
    x_validation = poly.fit_transform(x_validation)
    x_test = poly.fit_transform(x_test)
    input_parameters = poly.get_feature_names_out(input_parameters).tolist()

In [ ]:
## Select Scaler to use (Only one!)
dm.use_min_max_scaler = True
dm.use_standard_scaler = False
dm.scale_y_data = True
# [dm.inverse_min_max_scaler(dm.min_max_scaler(i, dm.x_min, dm.x_max), dm.x_min, dm.x_max)
#  for i in [0.01, 0.1, 1, 10]]
# [dm.inverse_standard_scaler(dm.standard_scaler(i, dm.x_min, dm.x_max), dm.x_min, dm.x_max)
#  for i in [0.01, 0.1, 1, 10]]
## Standard Scaler
dm.x_mean = np.mean(x_train)
dm.x_std = np.std(x_train)
dm.y_mean = np.mean(y_train)
dm.y_std = np.std(y_train)
## Min Max scaler
dm.x_min = np.min(x_train)
dm.x_max = np.max(x_train)
if dm.scale_y_data:
    dm.y_min = np.min(y_train)
    dm.y_max = np.max(y_train)
else:
    dm.y_min = np.array(0, dtype=np.float32)
    dm.y_max = np.array(1, dtype=np.float32)

In [ ]:
if dm.use_min_max_scaler:
    x_train = dm.min_max_scaler(x_train, dm.x_min, dm.x_max)
    x_validation = dm.min_max_scaler(x_validation, dm.x_min, dm.x_max)
    x_test = dm.min_max_scaler(x_test, dm.x_min, dm.x_max)
    if dm.scale_y_data:
        y_train = dm.min_max_scaler(y_train, dm.y_min, dm.y_max)
        y_validation = dm.min_max_scaler(y_validation, dm.y_min, dm.y_max)
        y_test = dm.min_max_scaler(y_test, dm.y_min, dm.y_max)
elif dm.use_standard_scaler:
    x_train = dm.standard_scaler(x_train, dm.x_mean, dm.x_std)
    x_test = dm.standard_scaler(x_test, dm.x_mean, dm.x_std)
    x_validation = dm.standard_scaler(x_validation, dm.x_mean, dm.x_std)
    if dm.scale_y_data:
        y_train = dm.standard_scaler(y_train, dm.y_mean, dm.y_std)
        y_validation = dm.standard_scaler(y_validation, dm.y_mean, dm.y_std)
        y_test = dm.standard_scaler(y_test, dm.y_mean, dm.y_std)

### Regression

In [ ]:
# Everything is logged in mlflow, create a new experiment
experiment_name = (
    name_of_this_run
    + "_"
    + "MISO_Linear_"
    + input_parameters_name
    + "_"
    + output_parameters_name
)
mlflow_exp = mlflow.set_experiment(experiment_name=experiment_name)

In [ ]:
with mlflow.start_run(experiment_id=mlflow_exp.experiment_id, run_name="Linear"):
    mlflow.sklearn.autolog()

    mlflow.log_param("arrhenius_correction", dm.use_arrhenius_correction)
    mlflow.log_param(
        "arrhenius_correction_with_factor", dm.use_arrhenius_correction_with_factor
    )
    mlflow.log_param("min_max_scaler", dm.use_min_max_scaler)
    mlflow.log_param("standard_scaler", dm.use_standard_scaler)
    mlflow.log_param("dm.scale_y_data", dm.scale_y_data)

    model = LinearRegression()

    model.fit(x_train, y_train)
    model.score(x_validation, y_validation)

    train_maxae_temp = dm.evaluate_max_abs_error(model, x_train, y_train)
    validation_maxae_temp = dm.evaluate_max_abs_error(model, x_validation, y_validation)
    test_maxae_temp = dm.evaluate_max_abs_error(model, x_test, y_test)
    train_mse_temp = dm.evaluate_mse(model, x_train, y_train)
    validation_mse_temp = dm.evaluate_mse(model, x_validation, y_validation)
    test_mse_temp = dm.evaluate_mse(model, x_test, y_test)
    train_rmse_temp = dm.evaluate_rmse(model, x_train, y_train)
    validation_rmse_temp = dm.evaluate_rmse(model, x_validation, y_validation)
    test_rmse_temp = dm.evaluate_rmse(model, x_test, y_test)

    mlflow.log_metric("train_maxae_temp", train_maxae_temp)
    mlflow.log_metric("validation_maxae_temp", validation_maxae_temp)
    mlflow.log_metric("test_maxae_temp", test_maxae_temp)
    mlflow.log_metric("train_mse_temp", train_mse_temp)
    mlflow.log_metric("validation_mse_temp", validation_mse_temp)
    mlflow.log_metric("test_mse_temp", test_mse_temp)
    mlflow.log_metric("train_rmse_temp", train_rmse_temp)
    mlflow.log_metric("validation_rmse_temp", validation_rmse_temp)
    mlflow.log_metric("test_rmse_temp", test_rmse_temp)

    merged_params = {"log_plot_type": ".svg"}
    dm.plot_diag_during_fitting(
        model,
        name_of_this_run,
        output_parameters,
        x_test,
        x_train,
        x_validation,
        data_set,
        train_rmse_temp,
        validation_rmse_temp,
        test_rmse_temp,
        merged_params,
    )

    mlflow.log_metric(
        "std_rmse", np.std([train_rmse_temp, validation_rmse_temp, test_rmse_temp])
    )
    mlflow.log_metric(
        "max_rmse", np.max([train_rmse_temp, validation_rmse_temp, test_rmse_temp])
    )
    mlflow.log_metric(
        "std_times_max_rmse",
        np.std([train_rmse_temp, validation_rmse_temp, test_rmse_temp])
        * np.max([train_rmse_temp, validation_rmse_temp, test_rmse_temp]),
    )

    weighted_fit_result = np.max(
        [train_rmse_temp, validation_rmse_temp, test_rmse_temp]
    )

    mlflow.log_metric("weighted_fit_result", weighted_fit_result)

In [ ]:
formula = "\n".join(
    [
        f"{param} = {intercept:.3f} + "
        + " + ".join(
            [f"{coef:.3f}*{feat}" for coef, feat in zip(coef_vector, input_parameters)]
        )
        for param, intercept, coef_vector in zip(
            output_parameters, model.intercept_, model.coef_
        )
    ]
)
print(formula)

### Select the best Model and validate

In [ ]:
name_of_this_run_eval = name_of_this_run

destination_filepath = r"./data/eis_datasets/" + name_of_this_run_eval + ".parquet"
df_eval = pd.read_parquet(destination_filepath)

In [ ]:
# change if needed
input_parameters_eval = input_parameters_backup
input_parameters_name_eval = input_parameters_name
output_parameters_eval = output_parameters
output_parameters_name_eval = output_parameters_name

test_labels_eval = test_labels
output_intervals_for_test_eval = output_intervals_for_test
polynomial_degree_eval = polynomial_degree

In [ ]:
data_set_eval = dm.get_set(
    df_eval,
    output_parameters_eval,
    feature_keys=input_parameters_eval,
    validation_split=0.2,
    output_intervals_for_test=output_intervals_for_test_eval,
    label_for_test_intervals=test_labels_eval,
    label_name=output_parameters_name_eval,
)
x_train_eval, y_train_eval = data_set_eval["train"]
x_validation_eval, y_validation_eval = data_set_eval["validation"]
x_test_eval, y_test_eval = data_set_eval["test"]

x_train_eval = np.float32(x_train_eval)
y_train_eval = np.float32(y_train_eval)
x_validation_eval = np.float32(x_validation_eval)
y_validation_eval = np.float32(y_validation_eval)
x_test_eval = np.float32(x_test_eval)
y_test_eval = np.float32(y_test_eval)

In [ ]:
if dm.use_arrhenius_correction:
    x_train_eval = dm.arrhenius_correction(x_train_eval)
    x_validation_eval = dm.arrhenius_correction(x_validation_eval)
    x_test_eval = dm.arrhenius_correction(x_test_eval)

if polynomial_degree_eval > 1:
    poly = PolynomialFeatures(
        degree=polynomial_degree_eval, include_bias=False, interaction_only=False
    )
    x_train_eval = poly.fit_transform(x_train_eval)
    x_validation_eval = poly.fit_transform(x_validation_eval)
    x_test_eval = poly.fit_transform(x_test_eval)
    input_parameters_eval = poly.get_feature_names_out(input_parameters_eval).tolist()

if dm.use_min_max_scaler:
    x_train_eval = dm.min_max_scaler(x_train_eval, dm.x_min, dm.x_max)
    x_validation_eval = dm.min_max_scaler(x_validation_eval, dm.x_min, dm.x_max)
    x_test_eval = dm.min_max_scaler(x_test_eval, dm.x_min, dm.x_max)
    if dm.scale_y_data:
        y_train_eval = dm.min_max_scaler(y_train_eval, dm.y_min, dm.y_max)
        y_validation_eval = dm.min_max_scaler(y_validation_eval, dm.y_min, dm.y_max)
        y_test_eval = dm.min_max_scaler(y_test_eval, dm.y_min, dm.y_max)
elif dm.use_standard_scaler:
    x_train_eval = dm.standard_scaler(x_train_eval, dm.x_mean, dm.x_std)
    x_test_eval = dm.standard_scaler(x_test_eval, dm.x_mean, dm.x_std)
    x_validation_eval = dm.standard_scaler(x_validation_eval, dm.x_mean, dm.x_std)
    if dm.scale_y_data:
        y_train_eval = dm.standard_scaler(y_train_eval, dm.y_mean, dm.y_std)
        y_validation_eval = dm.standard_scaler(y_validation_eval, dm.y_mean, dm.y_std)
        y_test_eval = dm.standard_scaler(y_test_eval, dm.y_mean, dm.y_std)

Open [http://127.0.0.1:1234](http://127.0.0.1:1234) to select a fitted model. If you click on it, you can extract the run ID. It could look like this: "ad26474e8c324f84906c9fc501928cae"

In [ ]:
# you can choose a specific model
# logged_model = 'ad26474e8c324f84906c9fc501928cae'
# or just load the best model
client = mlflow.tracking.MlflowClient()
runs = client.search_runs(
    experiment_ids=[mlflow.get_experiment_by_name(experiment_name).experiment_id],
    order_by=["metrics.max_rmse"],
    max_results=1,
)
logged_model = runs[0].info.run_id

# Load model as a Sklearn.
run_eval = mlflow.get_run(logged_model)
loaded_model = mlflow.sklearn.load_model(run_eval.info.artifact_uri + "/model/")

train_rmse_temp = dm.evaluate_rmse(loaded_model, x_train_eval, y_train_eval)
print("Train RMSE: " + str(train_rmse_temp))
validation_rmse_temp = dm.evaluate_rmse(
    loaded_model, x_validation_eval, y_validation_eval
)
print("Validation RMSE: " + str(validation_rmse_temp))
test_rmse_temp = dm.evaluate_rmse(loaded_model, x_test_eval, y_test_eval)
print("Test RMSE: " + str(test_rmse_temp))

formula = "\n".join(
    [
        f"{param} = {intercept:.3f} + "
        + " + ".join(
            [f"{coef:.3f}*{feat}" for coef, feat in zip(coef_vector, input_parameters)]
        )
        for param, intercept, coef_vector in zip(
            output_parameters_eval, loaded_model.intercept_, loaded_model.coef_
        )
    ]
)
print(formula)

unique_model_name = (
    experiment_name
    + "_"
    + mlflow.get_experiment_by_name(experiment_name).experiment_id
    + "_"
    + logged_model
)

### Plot Results

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10 * eisplot.cm, 10 * eisplot.cm))

cell_list_train = list(set(data_set_eval["df_train"].index.get_level_values(0)))
y_pred_train_eval = loaded_model.predict(x_train_eval)
y_pred_train_eval = y_pred_train_eval.ravel()

cell_list_validation = list(
    set(data_set_eval["df_validation"].index.get_level_values(0))
)
y_pred_validation_eval = loaded_model.predict(x_validation_eval)
y_pred_validation_eval = y_pred_validation_eval.ravel()

cell_list_test = list(set(data_set_eval["df_test"].index.get_level_values(0)))
y_pred_test_eval = loaded_model.predict(x_test_eval)
y_pred_test_eval = y_pred_test_eval.ravel()

if dm.scale_y_data:
    if dm.use_min_max_scaler:
        y_pred_train_eval = dm.inverse_min_max_scaler(
            y_pred_train_eval, dm.y_min, dm.y_max
        )
        y_pred_validation_eval = dm.inverse_min_max_scaler(
            y_pred_validation_eval, dm.y_min, dm.y_max
        )
        y_pred_test_eval = dm.inverse_min_max_scaler(
            y_pred_test_eval, dm.y_min, dm.y_max
        )
        y_train_plot = dm.inverse_min_max_scaler(y_train_eval, dm.y_min, dm.y_max)
        y_validation_plot = dm.inverse_min_max_scaler(
            y_validation_eval, dm.y_min, dm.y_max
        )
        y_test_plot = dm.inverse_min_max_scaler(y_test_eval, dm.y_min, dm.y_max)
    elif dm.use_standard_scaler:
        y_pred_train_eval = dm.inverse_standard_scaler(
            y_pred_train_eval, dm.y_mean, dm.y_std
        )
        y_pred_validation_eval = dm.inverse_standard_scaler(
            y_pred_validation_eval, dm.y_mean, dm.y_std
        )
        y_pred_test_eval = dm.inverse_standard_scaler(
            y_pred_test_eval, dm.y_mean, dm.y_std
        )
        y_train_plot = dm.inverse_standard_scaler(y_train_eval, dm.y_mean, dm.y_std)
        y_validation_plot = dm.inverse_standard_scaler(
            y_validation_eval, dm.y_mean, dm.y_std
        )
        y_test_plot = dm.inverse_standard_scaler(y_test_eval, dm.y_mean, dm.y_std)
else:
    y_train_plot = y_train_eval
    y_validation_plot = y_validation_eval
    y_test_plot = y_test_eval

fig, ax = eisplot.setup_scatter(
    data_set,
    test_rmse_temp,
    title=False,
    legend=False,
    fig=fig,
    ax=ax,
    ax_xlabel=False,
    ax_ylabel=False,
    subplots_adjust=True,
    add_trendline=True,
    label="",
)
ax.plot(
    y_train_plot,
    y_pred_train_eval,
    ".",
    color=eisplot.rwth_colors.colors[("petrol", 100)],
    alpha=0.5,
)
ax.plot(
    y_validation_plot,
    y_pred_validation_eval,
    "1",
    color=eisplot.rwth_colors.colors[("turqoise", 100)],
    alpha=0.5,
)
ax.plot(
    y_test_plot,
    y_pred_test_eval,
    "2",
    color=eisplot.rwth_colors.colors[("blue", 100)],
    alpha=0.5,
)

ax.set_xlabel("actual: Output")
ax.set_ylabel("predicted: Output")

legend_elements = [
    mpl.lines.Line2D(
        [0], [0], color=eisplot.rwth_colors.colors[("green", 100)], label="ideal"
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker=".",
        linestyle="",
        color=eisplot.rwth_colors.colors[("petrol", 100)],
        label="train",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="1",
        linestyle="",
        color=eisplot.rwth_colors.colors[("turqoise", 100)],
        label="validation",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="2",
        linestyle="",
        color=eisplot.rwth_colors.colors[("blue", 100)],
        label="test",
        alpha=0.5,
    ),
]
ax.legend(handles=legend_elements, loc="best", scatterpoints=1, prop={"size": 8})
fig.subplots_adjust(bottom=0.14, left=0.19)

### Convert, Export, Test and Validate with ONNX

In [ ]:
def convert_linear_regression(scope, operator, container):
    from skl2onnx.proto import onnx_proto
    from skl2onnx.common._apply_operation import apply_add, apply_matmul

    op = operator.raw_operator

    coef = op.coef_
    intercept = op.intercept_

    if coef.ndim == 1:
        coef = coef.reshape(-1, 1)
    else:
        coef = coef.T

    intercept = intercept.flatten()
    coef = coef.astype(float)
    intercept = intercept.astype(float)
    coef_name = scope.get_unique_variable_name("coef")
    intercept_name = scope.get_unique_variable_name("intercept")

    container.add_initializer(
        coef_name, onnx_proto.TensorProto.FLOAT, coef.shape, coef.flatten()
    )
    container.add_initializer(
        intercept_name, onnx_proto.TensorProto.FLOAT, intercept.shape, intercept
    )
    matmul_output_name = scope.get_unique_variable_name("matmul_output")
    apply_matmul(
        scope, [operator.inputs[0].full_name, coef_name], matmul_output_name, container
    )
    apply_add(
        scope,
        [matmul_output_name, intercept_name],
        operator.outputs[0].full_name,
        container,
    )


update_registered_converter(
    LinearRegression,
    "SklearnLinearRegression",
    calculate_linear_regressor_output_shapes,
    convert_linear_regression,
)

In [ ]:
input_type = [("float_input", FloatTensorType([None, len(x_train[0])]))]
onnx_filename = "microcontroller_eis_network/onnx_export/" + unique_model_name + ".onnx"
onx = convert_sklearn(loaded_model, initial_types=input_type)
with open(onnx_filename, "wb") as f:
    f.write(onx.SerializeToString())

In [ ]:
sess = rt.InferenceSession(onnx_filename, providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: x_test_eval})[0]
if dm.scale_y_data:
    if dm.use_min_max_scaler:
        pred_onx = dm.inverse_min_max_scaler(pred_onx, dm.y_min, dm.y_max)
        y_test_eval_ref = dm.inverse_min_max_scaler(y_test_eval, dm.y_min, dm.y_max)
    elif dm.use_standard_scaler:
        pred_onx = dm.inverse_standard_scaler(pred_onx, dm.y_mean, dm.y_std)
        y_test_eval_ref = dm.inverse_standard_scaler(y_test_eval, dm.y_mean, dm.y_std)
else:
    y_test_eval_ref = y_test_eval.copy()

diff = pred_onx.ravel() - y_test_eval_ref.ravel()
print(np.max(np.abs(diff)))
print(np.mean(diff))
print(np.std(diff))
print(np.sqrt(np.mean((diff) ** 2)))

In [ ]:
x_test_eval_float32 = x_test_eval.astype(np.float32)

pred_sklearn = loaded_model.predict(x_test_eval_float32)
pred_onx = sess.run([label_name], {input_name: x_test_eval_float32})[0].ravel()

if dm.scale_y_data:
    if dm.use_min_max_scaler:
        pred_sklearn = dm.inverse_min_max_scaler(pred_sklearn, dm.y_min, dm.y_max)
        pred_onx = dm.inverse_min_max_scaler(pred_onx, dm.y_min, dm.y_max)
    elif dm.use_standard_scaler:
        pred_sklearn = dm.inverse_standard_scaler(pred_sklearn, dm.y_mean, dm.y_std)
        pred_onx = dm.inverse_standard_scaler(pred_onx, dm.y_mean, dm.y_std)

diff = pred_sklearn.ravel() - pred_onx.ravel()
print("Max difference between scikit-learn and ONNX predictions:", np.max(np.abs(diff)))
print("Mean difference between scikit-learn and ONNX predictions:", np.mean(diff))
print("RMSE of differences:", np.sqrt(np.mean(diff**2)))

### Export the test data for the microcontroller

In [ ]:
header_path = "microcontroller_eis_network/Core/Inc/"
dm.create_test_header_file(
    data_set_eval,
    header_path,
    unique_model_name,
    polynomial_degree=polynomial_degree_eval,
)

### Further comparison of different fits

In [ ]:
experiment_list = pd.DataFrame()
experiment_id = mlflow_exp.experiment_id
experiment_id

for exp in mlflow.search_experiments():
    if exp.experiment_id == experiment_id:
        experiment_tmp = mlflow.search_runs(experiment_ids=experiment_id)
        experiment_list = pd.concat([experiment_list, experiment_tmp])

experiment_list = experiment_list.reset_index(drop=True)
experiment_list.head()

In [ ]:
scatter_values = [
    experiment_list["metrics.train_maxae_temp"].values,
    experiment_list["metrics.validation_maxae_temp"].values,
    experiment_list["metrics.test_maxae_temp"].values,
    experiment_list["metrics.train_mse_temp"].values,
    experiment_list["metrics.validation_mse_temp"].values,
    experiment_list["metrics.test_mse_temp"].values,
    experiment_list["metrics.train_rmse_temp"].values,
    experiment_list["metrics.validation_rmse_temp"].values,
    experiment_list["metrics.test_rmse_temp"].values,
]
df_experiment = pd.DataFrame(
    np.transpose(scatter_values),
    columns=[
        "Train MAXAE in K",
        "Validation MAXAE in K",
        "Test MAXAE in K",
        "Train MSE in K^2",
        "Validation MSE in K^2",
        "Test MSE in K^2",
        "Train RMSE in K",
        "Validation RMSE in K",
        "Test RMSE in K",
    ],
)

In [ ]:
destination_filepath = r"./mlruns/" + unique_model_name + ".parquet"
experiment_list.to_parquet(destination_filepath, compression="gzip", index=True)